In [1]:
from sdo.sood.data.sdo_ml_v2_dataset import SDOMLv2DataModule, SDOMLv2NumpyDataset

In [2]:
from pathlib import Path
import numpy as np
import os 
import sunpy
from sunpy.visualization.colormaps import cm
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import torchvision
import torch
import torchvision.transforms.functional as F

#inspect an image

#Channels that correspond to HMI Magnetograms 
HMI_WL = ['Bx','By','Bz']
#A colormap for visualizing HMI
HMI_CM = LinearSegmentedColormap.from_list("bwrblack", ["#0000ff","#000000","#ff0000"])

def channel_to_map(name):
    """Given channel name, return colormap"""
    return HMI_CM if name in HMI_WL else cm.cmlist.get('sdoaia%d' % int(name))

def get_clip(X,name):
    """Given an image and the channel name, get the right clip"""
    return get_signed_pct_clip(X) if name in HMI_WL else get_pct_clip(X)

def get_pct_clip(X):
    """Return the 99.99th percentile"""
    return (0,np.quantile(X.ravel(),0.999))

def get_signed_pct_clip(X):
    """Return the 99.99th percentile by magnitude, but symmetrize it so 0 is in the middle"""
    v = np.quantile(np.abs(X.ravel()),0.999)
    return (-v,v)

def vis(X, cm, clip):
    """Given image, colormap, and a clipping, visualize results"""
    Xc = np.clip((X-clip[0])/(clip[1]-clip[0]),0,1)
    Xcv = cm(Xc)
    return (Xcv[:,:,:3]*255).astype(np.uint8)

def show_grid(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fix, axs = plt.subplots(figsize=(24,24), ncols=4, nrows=int(len(imgs)/4), squeeze=False)
    row = 0
    for i, img in enumerate(imgs):
        col = i % 4
        if i != 0 and i % 4 == 0:
            row = row + 1
        axs[row, col].imshow(img)
        axs[row, col].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])
    plt.show()

In [3]:
loader = SDOMLv2DataModule("fdl-sdoml-v2/sdomlv2_small.zarr/", channel="193A", train_start="2010-08-27 00:00:00", train_end="2010-08-28 23:59:59", test_start="2010-08-29 00:00:00", test_end="2010-08-29 23:59:59", train_val_split_temporal_chunk_size="1h", prefetch_factor=2).train_dataloader()


discovered the following zarr directory structure
/
 └── 2010
     ├── 131A (6135, 512, 512) float32
     ├── 1600A (6136, 512, 512) float32
     ├── 1700A (6135, 512, 512) float32
     ├── 171A (6135, 512, 512) float32
     ├── 193A (6135, 512, 512) float32
     ├── 211A (6136, 512, 512) float32
     ├── 304A (6134, 512, 512) float32
     ├── 335A (6135, 512, 512) float32
     └── 94A (6136, 512, 512) float32
filtering data between 2010-08-27 00:00:00 and 2010-08-28 23:59:59
found 478 images
discovered the following zarr directory structure
/
 └── 2010
     ├── 131A (6135, 512, 512) float32
     ├── 1600A (6136, 512, 512) float32
     ├── 1700A (6135, 512, 512) float32
     ├── 171A (6135, 512, 512) float32
     ├── 193A (6135, 512, 512) float32
     ├── 211A (6136, 512, 512) float32
     ├── 304A (6134, 512, 512) float32
     ├── 335A (6135, 512, 512) float32
     └── 94A (6136, 512, 512) float32
filtering data between 2010-08-29 00:00:00 and 2010-08-29 23:59:59
found 239 images
Sele

In [4]:
for batch_idx, samples in enumerate(loader):
    X, y = samples
    V = []
    for x in X:
        x = x.permute(1,2,0) # torch to pillow
        x = np.squeeze(x.numpy())
        v = vis(x, channel_to_map(193), get_clip(x, 171))
        V.append(Image.fromarray(v))
    show_grid(V)
    break

In [10]:
for key in y.keys():
    print("%s %s" % (key, y[key][0]))

ACS_CGT GT3
ACS_ECLP NO
ACS_MODE SCIENCE
ACS_SAFE NO
ACS_SUNP YES
AECDELAY tensor(1545)
AECMODE ON
AECTYPE tensor(0)
AGT1SVY tensor(0)
AGT1SVZ tensor(4)
AGT2SVY tensor(0)
AGT2SVZ tensor(-4)
AGT3SVY tensor(1)
AGT3SVZ tensor(2)
AGT4SVY tensor(0)
AGT4SVZ tensor(0)
AIAECENF tensor(1)
AIAECTI tensor(0)
AIAGP1 tensor(0)
AIAGP10 tensor(748)
AIAGP2 tensor(0)
AIAGP3 tensor(0)
AIAGP4 tensor(0)
AIAGP5 tensor(0)
AIAGP6 tensor(0)
AIAGP7 tensor(0)
AIAGP8 tensor(280)
AIAGP9 tensor(344)
AIAHFSN tensor(11035686)
AIASEN tensor(6)
AIAWVLEN tensor(3)
AICFGDL1 tensor(0)
AICFGDL2 tensor(24)
AICFGDL3 tensor(88)
AICFGDL4 tensor(236)
AIFCPS tensor(55)
AIFDBID tensor(241)
AIFILTYP tensor(0)
AIFOENFL tensor(1)
AIFRMLID tensor(2908)
AIFTSID tensor(40962)
AIFTSWTH tensor(0)
AIFWEN tensor(270)
AIHIS192 tensor(0)
AIHIS348 tensor(0)
AIHIS604 tensor(0)
AIHIS860 tensor(0)
AIHISMXB tensor(0)
AIMGFSN tensor(2)
AIMGOTS tensor(1661666441)
AIMGOTSS tensor(46096)
AIMGSHCE tensor(2900)
AIMGSHEN tensor(4)
AIMGTYP tensor(0)
AIM

In [ ]:
loader = SDOMLv2DataModule("fdl-sdoml-v2/sdomlv2_hmi_small.zarr/", channel="Bx", batch_size=16).train_dataloader()

In [ ]:
for batch_idx, samples in enumerate(loader):
    X, y = samples
    V = []
    for x in X:
        x = x.permute(1,2,0) # torch to pillow
        x = np.squeeze(x.numpy())
        v = vis(x, channel_to_map("Bx"), get_clip(x, "Bx"))
        V.append(Image.fromarray(v))
    show_grid(V)
    break